In [1]:
from utils.experimentorUtils import *
from utils.experimentor_base import *
from utils.experimentor_proteins import * 
from utils.experimentor_planetoid import * 
from utils.constants import *
from utils.configs import * 
from utils.utils import * 
from torch_sparse import SparseTensor


config = {
    'num_of_epochs': 500,
    'num_of_runs': 3,
    'patience_period': 30,
    'num_workers': 0,
    'force_cpu': False,
    'test_frequency': 1,
    'console_log_freq': 1,
    'do_train_tqdm_logging': False,
    'computationBefore': False,
    'batch_size': 256,
    'test_batch_size': 256,
    'lr': 0.005,
    'num_heads': 4,
    'hidden_size': 64,
    'dropout': 0.6,  
    "use_layer_norm": False,
    "use_batch_norm": False,
    "dataset_name": "PubMed",
    "model_type": "GAT-V1",
    "nbor_degree": 2,
    "sparse": True,
    "aggr_mode": None,
    "num_of_layers": 2,
    "adj_mode": "OneStep"
}

In [2]:
import networkx as nx
import statistics
from itertools import chain
# This includes the isolated node!

build_graph = True; 

exp = Experimentor_Planetoid(config)

edge_index = exp.edge_index
edge_index = edge_index.set_diag(0)
print("Num_Nodes:", exp.num_nodes, "\n")
all_es = multi_step_sparse(edge_index, 5, exp.num_nodes)

for i, e in enumerate(all_es):
    if i+1 > 2:
        print("k=", i+1)
        print("\tSum:", e.to_dense().sum())
        edges = SparseTensor.to_torch_sparse_coo_tensor(e)._indices()

        if build_graph:
            edges_f = [(edge.item(), edges[1][i].item()) for i, edge in enumerate(edges[0])]
            G = nx.Graph(edges_f)
            print("\tNum_Edges", G.number_of_edges())
            path_lengths = (x.values() for x in dict(nx.shortest_path_length(G)).values())
            avg_length = statistics.mean(chain.from_iterable(path_lengths))
            print("\tAvg Shortest Path Length: ", avg_length)
